In [1]:
!pip install minsearch


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import json

In [5]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
import minsearch
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
SELECT * WHERE course = "data-engineering-zoomcamp"; 

In [10]:
q = "the course has already started, can I still enroll?"

In [14]:
index.fit(documents)

In [16]:
boost = {"question": 3.0, "section": 0.5}
results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [17]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

Calling an API to pass the context. The context is the set of documents we retrieve from the index and are related to the query.

In [22]:
from dotenv import load_dotenv
load_dotenv()

import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [23]:
q

'the course has already started, can I still enroll?'

In [24]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": q,
        }
    ],
    model="llama-3.3-70b-versatile",
)

In [28]:
response.choices[0].message.content

"It's possible to enroll in a course even if it has already started, but it depends on the specific course, institution, and their policies. Here are a few things to consider:\n\n1. **Check with the course instructor or administrator**: Reach out to the instructor or administrator of the course to ask if late enrollment is allowed. They may have specific requirements or restrictions for students who join after the start date.\n2. **Late enrollment deadlines**: Some institutions have specific deadlines for late enrollment, so be sure to check if you're still within the allowed timeframe.\n3. **Missed material**: If you enroll late, you may have missed some of the initial course material, lectures, or assignments. You'll need to catch up on any missed work and make arrangements to complete assignments or quizzes that you've missed.\n4. **Academic impact**: Enrolling late may impact your ability to fully participate in the course or affect your grades, especially if you've missed importan

In [34]:
prompt_template = """
You're a course teaching assistant. Answer the Question based on the Context. Use only the facts from the Cntext
If the Context doesn;t contain the answer, output NONE

Question: {question}

Context: {context}
""".strip()

In [43]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [45]:
prompt = prompt_template.format(question=q, context=context).strip()

In [46]:
print(prompt)

You're a course teaching assistant. Answer the Question based on the Context. Use only the facts from the Cntext
If the Context doesn;t contain the answer, output NONE

Question: the course has already started, can I still enroll?

Context: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-relate

In [47]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)

In [48]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks."